In [ ]:
from google.colab import files
import json

# Upload your analysis prompt file
print("📁 Upload your analysis_prompt_20251120_002408.txt file:")
uploaded = files.upload()

# Read the prompt
prompt_file = list(uploaded.keys())[0]
with open(prompt_file, 'r', encoding='utf-8') as f:
    analysis_prompt = f.read()

print(f"✅ Loaded prompt: {len(analysis_prompt)} characters")

📁 Upload your analysis_prompt_20251120_002408.txt file:


Saving analysis_prompt_20251120_002408.txt to analysis_prompt_20251120_002408 (1).txt
✅ Loaded prompt: 547991 characters


In [ ]:
!pip install -q anthropic google-generativeai groq together huggingface_hub requests

In [ ]:
import os
from google.colab import userdata

# Store your API keys in Colab Secrets (left sidebar → 🔑 Secrets)
# Or enter them directly here (less secure):
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')        # Gemini
GROQ_API_KEY = userdata.get('GROQ_API_KEY')            # Groq (Free)
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY') # OpenRouter
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY') # HuggingFace

print("🔑 API Keys loaded!")

🔑 API Keys loaded!


In [ ]:
!pip install -q anthropic google-generativeai groq together huggingface_hub requests

In [ ]:
import google.generativeai as genai
from groq import Groq
from huggingface_hub import InferenceClient
import requests
import json
from datetime import datetime
import time
from google.colab import userdata, files

# Configure APIs
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')

# Initialize clients
genai.configure(api_key=GOOGLE_API_KEY)
groq_client = Groq(api_key=GROQ_API_KEY)
hf_client = InferenceClient(token=HUGGINGFACE_API_KEY)

results = {}
errors = {}

print("🏥 MULTI-LLM HOSPITAL ANALYSIS STARTING...\n")
print("="*60)

# Function to create a smaller, focused prompt
def create_focused_prompt(full_data):
    """Create a smaller prompt by summarizing data"""

    # Extract just the key information
    hospitals = list(full_data['data'].keys())

    # Build a condensed version with sample reviews only
    condensed_data = {}
    for hospital, data in full_data['data'].items():
        # Take only first 10 reviews and 5 news items as samples
        condensed_data[hospital] = {
            'review_count': data['review_count'],
            'news_count': data['news_count'],
            'sample_reviews': data['reviews'][:10],  # First 10 reviews
            'sample_news': data['news'][:5]  # First 5 news items
        }

    prompt = f"""I have collected review and news data for {len(hospitals)} hospitals in Delhi:
{', '.join(hospitals)}

DATA SUMMARY:
{chr(10).join([f"{h}: {condensed_data[h]['review_count']} reviews, {condensed_data[h]['news_count']} news" for h in hospitals])}

SAMPLE DATA (showing first 10 reviews and 5 news per hospital):
{json.dumps(condensed_data, indent=2)}

Based on this sample data, provide:

A. OVERALL RANKING (1-{len(hospitals)}):
- Rank hospitals best to worst
- Give each a score out of 10
- Justify with specific review quotes in [brackets]

B. TOP 3 STRENGTHS & WEAKNESSES for each hospital:
- Use actual review quotes as evidence

C. BEST HOSPITAL FOR:
- Emergency care
- Planned surgery
- Budget patients
- Cancer treatment
- Heart disease
- Orthopedics

D. KEY RECOMMENDATIONS:
- Which hospitals to prefer for specific needs
- Which to avoid and why

Be specific and quote actual reviews in [brackets]."""

    return prompt

# Load your full data
print("📁 Upload your combined hospital data JSON file:")
uploaded = files.upload()
data_file = list(uploaded.keys())[0]

with open(data_file, 'r', encoding='utf-8') as f:
    full_hospital_data = json.load(f)

# Create focused prompt
analysis_prompt = create_focused_prompt(full_hospital_data)
print(f"\n✅ Created focused prompt: ~{len(analysis_prompt)} characters")
print(f"Estimated tokens: ~{len(analysis_prompt.split())}")

# 1. GEMINI 1.5 FLASH (Fallback to older model with higher quota)
try:
    print("\n🤖 Running: Gemini 1.5 Flash...")
    start = time.time()

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(analysis_prompt)

    results['gemini_flash'] = {
        'response': response.text,
        'time': time.time() - start,
        'model': 'Gemini 1.5 Flash (Google)'
    }
    print(f"✅ Gemini Flash completed in {results['gemini_flash']['time']:.2f}s")

except Exception as e:
    errors['gemini_flash'] = str(e)
    print(f"❌ Gemini Flash failed: {e}")

# 2. GROQ - Llama 3.1 70B Versatile (Updated model)
try:
    print("\n🤖 Running: Groq Llama 3.1 70B...")
    start = time.time()

    response = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": analysis_prompt}],
        temperature=0.7,
        max_tokens=4000
    )

    results['groq_llama_70b'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.1 70B (Groq)'
    }
    print(f"✅ Groq Llama 70B completed in {results['groq_llama_70b']['time']:.2f}s")

except Exception as e:
    errors['groq_llama_70b'] = str(e)
    print(f"❌ Groq Llama 70B failed: {e}")

# 3. GROQ - Llama 3.2 90B (Newer model)
try:
    print("\n🤖 Running: Groq Llama 3.2 90B...")
    start = time.time()

    response = groq_client.chat.completions.create(
        model="llama-3.2-90b-text-preview",
        messages=[{"role": "user", "content": analysis_prompt}],
        temperature=0.7,
        max_tokens=4000
    )

    results['groq_llama_90b'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.2 90B (Groq)'
    }
    print(f"✅ Groq Llama 90B completed in {results['groq_llama_90b']['time']:.2f}s")

except Exception as e:
    errors['groq_llama_90b'] = str(e)
    print(f"❌ Groq Llama 90B failed: {e}")

# 4. OPENROUTER - Claude 3.5 Haiku
try:
    print("\n🤖 Running: OpenRouter Claude 3.5 Haiku...")
    start = time.time()

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com/yourusername",
            "X-Title": "Hospital Analysis"
        },
        json={
            "model": "anthropic/claude-3.5-haiku",
            "messages": [{"role": "user", "content": analysis_prompt}],
            "max_tokens": 4000
        },
        timeout=120
    )

    response.raise_for_status()
    response_data = response.json()

    results['openrouter_claude'] = {
        'response': response_data['choices'][0]['message']['content'],
        'time': time.time() - start,
        'model': 'Claude 3.5 Haiku (OpenRouter)'
    }
    print(f"✅ OpenRouter Claude completed in {results['openrouter_claude']['time']:.2f}s")

except Exception as e:
    errors['openrouter_claude'] = str(e)
    print(f"❌ OpenRouter Claude failed: {e}")

# 5. OPENROUTER - Meta Llama 3.1 405B
try:
    print("\n🤖 Running: OpenRouter Llama 405B...")
    start = time.time()

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com/yourusername",
            "X-Title": "Hospital Analysis"
        },
        json={
            "model": "meta-llama/llama-3.1-405b-instruct",
            "messages": [{"role": "user", "content": analysis_prompt}],
            "max_tokens": 4000
        },
        timeout=120
    )

    response.raise_for_status()
    response_data = response.json()

    results['openrouter_llama405b'] = {
        'response': response_data['choices'][0]['message']['content'],
        'time': time.time() - start,
        'model': 'Llama 3.1 405B (OpenRouter)'
    }
    print(f"✅ OpenRouter Llama 405B completed in {results['openrouter_llama405b']['time']:.2f}s")

except Exception as e:
    errors['openrouter_llama405b'] = str(e)
    print(f"❌ OpenRouter Llama 405B failed: {e}")

# 6. HUGGINGFACE - Qwen 2.5 72B
try:
    print("\n🤖 Running: HuggingFace Qwen 2.5 72B...")
    start = time.time()

    response = hf_client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "user", "content": analysis_prompt}],
        max_tokens=4000
    )

    results['hf_qwen'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Qwen 2.5 72B (HuggingFace)'
    }
    print(f"✅ HuggingFace Qwen completed in {results['hf_qwen']['time']:.2f}s")

except Exception as e:
    errors['hf_qwen'] = str(e)
    print(f"❌ HuggingFace Qwen failed: {e}")

# 7. HUGGINGFACE - Meta Llama 3.1 70B
try:
    print("\n🤖 Running: HuggingFace Llama 3.1 70B...")
    start = time.time()

    response = hf_client.chat_completion(
        model="meta-llama/Llama-3.1-70B-Instruct",
        messages=[{"role": "user", "content": analysis_prompt}],
        max_tokens=4000
    )

    results['hf_llama'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.1 70B (HuggingFace)'
    }
    print(f"✅ HuggingFace Llama completed in {results['hf_llama']['time']:.2f}s")

except Exception as e:
    errors['hf_llama'] = str(e)
    print(f"❌ HuggingFace Llama failed: {e}")

print("\n" + "="*60)
print("📊 ANALYSIS COMPLETE!")
print(f"✅ Successful: {len(results)} models")
print(f"❌ Failed: {len(errors)} models")

if errors:
    print("\n⚠️ Failed models:")
    for model, error in errors.items():
        print(f"  - {model}: {error[:100]}...")

# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_data = {
    'timestamp': timestamp,
    'successful_models': len(results),
    'failed_models': len(errors),
    'results': results,
    'errors': errors,
    'prompt_used': analysis_prompt
}

output_file = f'hospital_analysis_{timestamp}.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_file}")
files.download(output_file)

# Save individual markdown reports
for model_name, data in results.items():
    filename = f"{model_name}_analysis.md"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"# Hospital Analysis - {data['model']}\n\n")
        f.write(f"**Generated:** {timestamp}\n")
        f.write(f"**Processing Time:** {data['time']:.2f}s\n\n")
        f.write("---\n\n")
        f.write(data['response'])
    print(f"📄 Saved: {filename}")
    files.download(filename)

# Print preview
print("\n" + "="*60)
print("RESULTS PREVIEW")
print("="*60)
for model_name, data in results.items():
    print(f"\n🤖 {data['model']}")
    print(f"⏱️  {data['time']:.2f}s")
    print("-"*60)
    print(data['response'][:400] + "...\n")

🏥 MULTI-LLM HOSPITAL ANALYSIS STARTING...

📁 Upload your combined hospital data JSON file:


Saving all_hospitals_combined_20251120_002403.json to all_hospitals_combined_20251120_002403.json

✅ Created focused prompt: ~64140 characters
Estimated tokens: ~9591

🤖 Running: Gemini 1.5 Flash...


❌ Gemini Flash failed: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

🤖 Running: Groq Llama 3.1 70B...
❌ Groq Llama 70B failed: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

🤖 Running: Groq Llama 3.2 90B...
❌ Groq Llama 90B failed: Error code: 400 - {'error': {'message': 'The model `llama-3.2-90b-text-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use i

Model meta-llama/Llama-3.1-70B-Instruct is in staging mode for provider hyperbolic. Meant for test purposes only.


❌ HuggingFace Qwen failed: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/novita/v3/openai/chat/completions

🤖 Running: HuggingFace Llama 3.1 70B...
✅ HuggingFace Llama completed in 39.26s

📊 ANALYSIS COMPLETE!
✅ Successful: 2 models
❌ Failed: 5 models

⚠️ Failed models:
  - gemini_flash: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%2...
  - groq_llama_70b: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned...
  - groq_llama_90b: Error code: 400 - {'error': {'message': 'The model `llama-3.2-90b-text-preview` has been decommissio...
  - openrouter_llama405b: HTTPSConnectionPool(host='openrouter.ai', port=443): Read timed out. (read timeout=120)...
  - hf_qwen: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/novita/v3/openai/chat/comp...

💾 Results saved to: hospital_analysis_20251121_062544.json


...

<IPython.core.display.Javascript object>

📄 Saved: hf_llama_analysis.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


RESULTS PREVIEW

🤖 Claude 3.5 Haiku (OpenRouter)
⏱️  20.37s
------------------------------------------------------------
A. OVERALL RANKING (Out of 10):

1. AIIMS (8.5/10)
- Consistently positive reviews
- [Cutting edge technology]
- [Internationally renowned doctors]
- [High-quality care at reasonable prices]

2. Max (7/10)
- Mixed reviews with some excellent experiences
- [Professional and caring staff in some departments]
- Some serious patient care concerns

3. Apollo (6.5/10)
- Varied experiences
- [Good for in...


🤖 Llama 3.1 70B (HuggingFace)
⏱️  39.26s
------------------------------------------------------------
A. OVERALL RANKING (1-7):

1. AIIMS - Score: 8.5/10 [Reviews consistently mention excellent treatment, good doctors, and high-quality facilities. "Aiims Hospital, Madurai ensures that you get the highest quality of care and it has top-notch facilities and amenities."]

2. Max - Score: 7.5/10 [Reviews praise the hospital's cleanliness, professionalism, and care. Howeve